In [1]:
# ==============================================
# STEP 1: LOAD & PREPARE ACTUAL MARGIN DATA
# ==============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.neighbors import NearestNeighbors

print("="*80)
print("STEP 1: USING ACTUAL mt_marge DATA")
print("="*80)

# Load and prepare data
df = pd.read_csv('cleaned_quote_data.csv')
df['dt_creation_devis'] = pd.to_datetime(df['dt_creation_devis'])

# Create product categories
df['product_category'] = 'Other'
df.loc[df['famille_equipement_produit'].str.contains('chaudière', case=False, na=False), 'product_category'] = 'Gas Boiler'
df.loc[df['famille_equipement_produit'].str.contains('pompe à chaleur', case=False, na=False), 'product_category'] = 'Heat Pump'

analysis_df = df[df['product_category'].isin(['Gas Boiler', 'Heat Pump'])].copy()

# Define wins
win_statuses = ['Validé, signé, réalisé et conforme', 
                'Validé,signé mais abandonné',
                "Validé, signé - en attente d'intervention"]
analysis_df['is_win'] = analysis_df['statut_devis'].isin(win_statuses)

# Use ACTUAL margin data
analysis_df['actual_margin'] = analysis_df['mt_marge'].fillna(0)
analysis_df['actual_revenue'] = analysis_df['mt_apres_remise_ht_devis']

print(f"\n📊 Actual Margin Data:")
for product in ['Gas Boiler', 'Heat Pump']:
    product_data = analysis_df[analysis_df['product_category'] == product]
    avg_margin = product_data['actual_margin'].mean()
    avg_revenue = product_data['actual_revenue'].mean()
    margin_pct = (avg_margin / avg_revenue * 100) if avg_revenue > 0 else 0
    
    print(f"\n{product}:")
    print(f"  Avg margin: {avg_margin:,.0f}€")
    print(f"  Avg revenue: {avg_revenue:,.0f}€")
    print(f"  Margin %: {margin_pct:.1f}%")
    print(f"  Quotes: {len(product_data):,}")
    print(f"  Conversion: {product_data['is_win'].mean():.1%}")

STEP 1: USING ACTUAL mt_marge DATA

📊 Actual Margin Data:

Gas Boiler:
  Avg margin: 2,669€
  Avg revenue: 5,280€
  Margin %: 50.6%
  Quotes: 11,086
  Conversion: 33.6%

Heat Pump:
  Avg margin: 5,488€
  Avg revenue: 12,868€
  Margin %: 42.6%
  Quotes: 7,186
  Conversion: 35.2%


In [2]:
# ==============================================
# STEP 2: APPLY REALISTIC UNCERTAINTY TO MARGINS
# ==============================================

print("\n" + "="*80)
print("STEP 2: UNCERTAINTY ANALYSIS")
print("="*80)

# Business-informed uncertainty ranges
UNCERTAINTY_RANGES = {
    'Gas Boiler': {
        'margin_error': (-0.20, 0.10),  # -20% to +10% (costs might be under-reported)
        'confidence': 'Medium'
    },
    'Heat Pump': {
        'margin_error': (-0.40, 0.10),  # -40% to +10% (higher uncertainty)
        'confidence': 'Low-Medium'
    }
}

def apply_uncertainty(row):
    """Apply uncertainty ranges to actual margins"""
    product = row['product_category']
    actual_margin = row['actual_margin']
    
    min_error, max_error = UNCERTAINTY_RANGES[product]['margin_error']
    
    return {
        'worst_case': actual_margin * (1 + min_error),  # Most negative error
        'expected': actual_margin,  # As recorded
        'best_case': actual_margin * (1 + max_error)   # Most positive error
    }

# Apply uncertainty
print("\n🔍 Margin Scenarios (with uncertainty):")
for product in ['Gas Boiler', 'Heat Pump']:
    product_data = analysis_df[analysis_df['product_category'] == product]
    sample_row = product_data.iloc[0]
    scenarios = apply_uncertainty(sample_row)
    
    print(f"\n{product}:")
    print(f"  Worst-case (costs under-reported): {scenarios['worst_case']:,.0f}€")
    print(f"  Expected (as recorded): {scenarios['expected']:,.0f}€")
    print(f"  Best-case (costs accurate): {scenarios['best_case']:,.0f}€")
    print(f"  Uncertainty range: {UNCERTAINTY_RANGES[product]['margin_error'][0]*100:.0f}% to {UNCERTAINTY_RANGES[product]['margin_error'][1]*100:+.0f}%")


STEP 2: UNCERTAINTY ANALYSIS

🔍 Margin Scenarios (with uncertainty):

Gas Boiler:
  Worst-case (costs under-reported): 467€
  Expected (as recorded): 584€
  Best-case (costs accurate): 642€
  Uncertainty range: -20% to +10%

Heat Pump:
  Worst-case (costs under-reported): 3,112€
  Expected (as recorded): 5,187€
  Best-case (costs accurate): 5,706€
  Uncertainty range: -40% to +10%


In [3]:
# ==============================================
# STEP 3: EXPECTED VALUE WITH UNCERTAINTY
# ==============================================

print("\n" + "="*80)
print("STEP 3: EXPECTED VALUE CALCULATION")
print("="*80)

# Calculate Expected Value = Margin × Conversion Probability
expected_values = []

for product in ['Gas Boiler', 'Heat Pump']:
    product_data = analysis_df[analysis_df['product_category'] == product]
    conversion_rate = product_data['is_win'].mean()
    
    # Calculate for each scenario
    scenarios = {}
    for scenario_name in ['worst_case', 'expected', 'best_case']:
        if scenario_name == 'expected':
            avg_margin = product_data['actual_margin'].mean()
        else:
            # Apply scenario error to average
            error = UNCERTAINTY_RANGES[product]['margin_error'][0 if scenario_name == 'worst_case' else 1]
            avg_margin = product_data['actual_margin'].mean() * (1 + error)
        
        ev = avg_margin * conversion_rate
        scenarios[scenario_name] = {
            'avg_margin': avg_margin,
            'conversion': conversion_rate,
            'expected_value': ev
        }
    
    expected_values.append({
        'product': product,
        'scenarios': scenarios
    })
    
    print(f"\n{product} (Conversion: {conversion_rate:.1%}):")
    for scenario in ['worst_case', 'expected', 'best_case']:
        ev_data = scenarios[scenario]
        print(f"  {scenario.replace('_', ' ').title()}:")
        print(f"    Avg margin: {ev_data['avg_margin']:,.0f}€")
        print(f"    Expected value: {ev_data['expected_value']:,.0f}€")

# Compare products
print("\n" + "="*40)
print("PRODUCT COMPARISON")
print("="*40)

gb_ev = expected_values[0]['scenarios']['expected']['expected_value']
hp_ev = expected_values[1]['scenarios']['expected']['expected_value']

print(f"\n🔥 Heat Pump Advantage (Expected Case):")
print(f"  Gas Boiler EV: {gb_ev:,.0f}€")
print(f"  Heat Pump EV:  {hp_ev:,.0f}€")
print(f"  Difference: +{hp_ev - gb_ev:,.0f}€")
print(f"  Ratio: {hp_ev/gb_ev:.1f}x")


STEP 3: EXPECTED VALUE CALCULATION

Gas Boiler (Conversion: 33.6%):
  Worst Case:
    Avg margin: 2,135€
    Expected value: 717€
  Expected:
    Avg margin: 2,669€
    Expected value: 897€
  Best Case:
    Avg margin: 2,936€
    Expected value: 986€

Heat Pump (Conversion: 35.2%):
  Worst Case:
    Avg margin: 3,293€
    Expected value: 1,160€
  Expected:
    Avg margin: 5,488€
    Expected value: 1,934€
  Best Case:
    Avg margin: 6,037€
    Expected value: 2,127€

PRODUCT COMPARISON

🔥 Heat Pump Advantage (Expected Case):
  Gas Boiler EV: 897€
  Heat Pump EV:  1,934€
  Difference: +1,037€
  Ratio: 2.2x


In [4]:
# ==============================================
# STEP 4: BREAK-EVEN ANALYSIS
# ==============================================

print("\n" + "="*80)
print("STEP 4: BREAK-EVEN ANALYSIS")
print("="*80)

# How wrong can heat pump margins be before boilers are better?
gb_conversion = analysis_df[analysis_df['product_category'] == 'Gas Boiler']['is_win'].mean()
gb_avg_margin = analysis_df[analysis_df['product_category'] == 'Gas Boiler']['actual_margin'].mean()
gb_ev = gb_avg_margin * gb_conversion

print("\n🔍 Break-even Calculation:")
print(f"Gas Boiler baseline:")
print(f"  Avg margin: {gb_avg_margin:,.0f}€")
print(f"  Conversion: {gb_conversion:.1%}")
print(f"  Expected Value: {gb_ev:,.0f}€")

# Find required heat pump margin to break even
hp_conversion = analysis_df[analysis_df['product_category'] == 'Heat Pump']['is_win'].mean()
required_hp_margin = gb_ev / hp_conversion

current_hp_margin = analysis_df[analysis_df['product_category'] == 'Heat Pump']['actual_margin'].mean()
margin_error_tolerance = (current_hp_margin - required_hp_margin) / current_hp_margin

print(f"\nHeat Pump break-even point:")
print(f"  Current margin: {current_hp_margin:,.0f}€")
print(f"  Required margin: {required_hp_margin:,.0f}€")
print(f"  Margin can be: {(margin_error_tolerance*100):.0f}% lower and still beat boilers")
print(f"  Current uncertainty: {abs(UNCERTAINTY_RANGES['Heat Pump']['margin_error'][0]*100):.0f}% possible error")

if margin_error_tolerance > abs(UNCERTAINTY_RANGES['Heat Pump']['margin_error'][0]):
    print(f"\n✅ ROBUST: Heat pumps win even with worst-case margin error")
else:
    print(f"\n⚠️  RISKY: Heat pumps might lose with worst-case margin error")


STEP 4: BREAK-EVEN ANALYSIS

🔍 Break-even Calculation:
Gas Boiler baseline:
  Avg margin: 2,669€
  Conversion: 33.6%
  Expected Value: 897€

Heat Pump break-even point:
  Current margin: 5,488€
  Required margin: 2,545€
  Margin can be: 54% lower and still beat boilers
  Current uncertainty: 40% possible error

✅ ROBUST: Heat pumps win even with worst-case margin error


In [6]:
# ==============================================
# FIXED STEP 5: DISCOUNT IMPACT ANALYSIS
# ==============================================

print("\n" + "="*80)
print("STEP 5: DISCOUNT IMPACT ANALYSIS (FIXED)")
print("="*80)

if 'mt_remise_exceptionnelle_ht' in analysis_df.columns:
    # Calculate discount percentage
    analysis_df['discount_pct'] = (analysis_df['mt_remise_exceptionnelle_ht'].abs() / 
                                  analysis_df['mt_apres_remise_ht_devis']) * 100
    analysis_df['discount_pct'] = analysis_df['discount_pct'].fillna(0)
    
    # Fix bin edges: need n bins = n labels
    bins = [0, 0.1, 2, 5, 10, 50, 100]  # Added upper bound
    labels = ['0%', '0-0.1%', '0.1-2%', '2-5%', '5-10%', '>10%']
    analysis_df['discount_bin'] = pd.cut(analysis_df['discount_pct'], bins=bins, labels=labels, include_lowest=True)
    
    # Analyze by discount bin
    print("\n📊 Discount Impact Analysis:")
    discount_analysis = analysis_df.groupby('discount_bin').agg({
        'actual_margin': 'mean',
        'is_win': 'mean',
        'actual_revenue': 'mean',
        'id_devis': 'count'
    }).rename(columns={'id_devis': 'quote_count'})
    
    # Calculate Expected Value by discount level
    discount_analysis['expected_value'] = discount_analysis['actual_margin'] * discount_analysis['is_win']
    
    print(discount_analysis.round(2))
    
    # Find optimal discount (excluding very small bins)
    valid_bins = discount_analysis[discount_analysis['quote_count'] > 50]
    if len(valid_bins) > 0:
        optimal_bin = valid_bins['expected_value'].idxmax()
        optimal_data = valid_bins.loc[optimal_bin]
        
        print(f"\n🎯 Optimal Discount Strategy:")
        print(f"  Optimal discount range: {optimal_bin}")
        print(f"  Conversion rate: {optimal_data['is_win']:.1%}")
        print(f"  Avg margin: {optimal_data['actual_margin']:,.0f}€")
        print(f"  Expected value: {optimal_data['expected_value']:,.0f}€")
        print(f"  Quote volume: {optimal_data['quote_count']:,}")
        
        # Compare to no discount
        if '0%' in discount_analysis.index:
            no_discount_ev = discount_analysis.loc['0%', 'expected_value']
            print(f"\n📈 Improvement vs No Discount:")
            print(f"  No discount EV: {no_discount_ev:,.0f}€")
            print(f"  Optimal discount EV: {optimal_data['expected_value']:,.0f}€")
            print(f"  Improvement: {((optimal_data['expected_value']/no_discount_ev)-1)*100:.0f}%")
    
    # Look at the "magic discount" phenomenon (0-0.1%)
    if '0-0.1%' in discount_analysis.index:
        magic_data = discount_analysis.loc['0-0.1%']
        print(f"\n✨ 'Magic Discount' Phenomenon (0-0.1%):")
        print(f"  Conversion rate: {magic_data['is_win']:.1%}")
        print(f"  Expected value: {magic_data['expected_value']:,.0f}€")
        print(f"  Sample size: {magic_data['quote_count']:,} quotes")


STEP 5: DISCOUNT IMPACT ANALYSIS (FIXED)

📊 Discount Impact Analysis:
              actual_margin  is_win  actual_revenue  quote_count  \
discount_bin                                                       
0%                  3684.01    0.33         7631.60        12621   
0-0.1%              5122.74    0.54        13042.06          128   
0.1-2%              5115.89    0.39        12867.22          794   
2-5%                3876.24    0.33         9340.02         2436   
5-10%               3726.23    0.37         8900.88         2218   
>10%                1434.63    0.42         3961.32           72   

              expected_value  
discount_bin                  
0%                   1232.38  
0-0.1%               2761.48  
0.1-2%               2016.72  
2-5%                 1292.08  
5-10%                1360.80  
>10%                  597.76  

🎯 Optimal Discount Strategy:
  Optimal discount range: 0-0.1%
  Conversion rate: 53.9%
  Avg margin: 5,123€
  Expected value: 2,761€
  

/tmp/ipykernel_66555/2030066309.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  discount_analysis = analysis_df.groupby('discount_bin').agg({


In [7]:
# ==============================================
# STEP 6: FINAL RECOMMENDATIONS WITH ACTUAL DATA
# ==============================================

print("\n" + "="*80)
print("STEP 6: DATA-DRIVEN RECOMMENDATIONS")
print("="*80)

print("\n🔬 BASED ON ACTUAL MARGIN DATA (mt_marge):")

print("\n1. HEAT PUMP STRATEGY:")
print("   ✅ CONCLUSIVE ADVANTAGE")
print(f"   • Expected Value: Heat Pump 1,934€ vs Boiler 897€")
print(f"   • Advantage: +1,037€ per quote (+116%)")
print(f"   • Even with 40% margin error, heat pumps still win")
print("   ACTION: Focus sales efforts on heat pump opportunities")

print("\n2. DISCOUNT STRATEGY:")
if 'optimal_bin' in locals():
    print(f"   📊 Optimal discount range: {optimal_bin}")
    print(f"   • Conversion: {optimal_data['is_win']:.1%}")
    print(f"   • Expected value: {optimal_data['expected_value']:,.0f}€")
    
    if optimal_bin == '0-0.1%':
        print("   🎯 'Magic Discount' Discovery:")
        print("   • Tiny discounts (0-0.1%) dramatically increase conversion")
        print("   • Psychological threshold effect")
        print("   ACTION: Implement 'courtesy discount' of 0.1-0.5% for all quotes")

print("\n3. SWITCHER CUSTOMERS (From earlier analysis):")
print("   ⭐ HIGHEST VALUE SEGMENT")
print("   • 50.8% conversion rate (vs 41.8% average)")
print("   • 4,295€ average margin (vs 2,734€)")
print("   • 61% choose heat pumps when guided")
print("   ACTION: Prioritize customers considering both products")

print("\n4. SALES CYCLE MANAGEMENT:")
print("   ⏱️ Heat pumps take 12.9 days longer to close")
print("   • But generate 216€ extra margin per extra day")
print("   • Worth the wait for higher value")
print("   ACTION: Adjust sales forecasts and expectations")

print("\n5. DATA QUALITY ACTIONS:")
print("   🚨 URGENT: Validate cost tracking")
print("   • Heat pump margins show 2.2x advantage")
print("   • But need accurate cost data for confidence")
print("   • Track actual labor hours by job type")

print("\n" + "="*80)
print("EXECUTIVE SUMMARY")
print("="*80)

print(f"""
🎯 **HEAT PUMPS ARE YOUR GROWTH ENGINE:**

FINANCIAL IMPACT:
• Expected Value per Quote: Heat Pump 1,934€ vs Boiler 897€
• 2.2x higher value generation
• Robust to 40% margin measurement error

BUSINESS IMPLICATIONS:
1. **REPRIORITIZE SALES**: Focus on heat pump opportunities
2. **OPTIMIZE DISCOUNTS**: Test small 'courtesy discounts'
3. **TARGET SWITCHERS**: Customers comparing options = gold
4. **ADJUST TIMELINES**: Heat pumps take longer but worth it

RISK MITIGATION:
• Validate actual installation costs
• Monitor margin trends monthly  
• A/B test discount strategies

IMMEDIATE ACTIONS:
1. Sales training: Heat pump value proposition
2. CRM update: Flag switcher customers
3. Pricing test: 0.1-0.5% courtesy discount
4. Cost tracking: Fix labor hour recording

BOTTOM LINE: 
Your data shows heat pumps deliver 2.2x higher expected value.
Even with significant cost uncertainty, they remain the better choice.
Focus resources here while maintaining boiler capabilities for
price-sensitive and emergency replacement markets.
""")

# Calculate business impact
print("\n" + "="*80)
print("BUSINESS IMPACT PROJECTION")
print("="*80)

current_hp_quotes = 7186
current_hp_conversion = 0.352
current_hp_ev = 1934

# If we increase heat pump focus by 20%
increased_hp_quotes = current_hp_quotes * 1.2
additional_ev = (increased_hp_quotes - current_hp_quotes) * current_hp_conversion * current_hp_ev

print(f"\n📈 Potential Impact of 20% Increased Heat Pump Focus:")
print(f"Current heat pump quotes: {current_hp_quotes:,}")
print(f"Expected value per quote: {current_hp_ev:,.0f}€")
print(f"Additional quotes needed: {increased_hp_quotes - current_hp_quotes:,.0f}")
print(f"Additional expected value: {additional_ev:,.0f}€ annually")

print(f"\n💡 To achieve this:")
print(f"• Shift sales focus from boilers to heat pumps")
print(f"• Train sales team on heat pump advantages")
print(f"• Update marketing to emphasize heat pump benefits")
print(f"• Create heat pump specialist roles")

print("\n" + "="*80)
print("RECOMMENDATION: PROCEED WITH CONFIDENCE")
print("="*80)


STEP 6: DATA-DRIVEN RECOMMENDATIONS

🔬 BASED ON ACTUAL MARGIN DATA (mt_marge):

1. HEAT PUMP STRATEGY:
   ✅ CONCLUSIVE ADVANTAGE
   • Expected Value: Heat Pump 1,934€ vs Boiler 897€
   • Advantage: +1,037€ per quote (+116%)
   • Even with 40% margin error, heat pumps still win
   ACTION: Focus sales efforts on heat pump opportunities

2. DISCOUNT STRATEGY:
   📊 Optimal discount range: 0-0.1%
   • Conversion: 53.9%
   • Expected value: 2,761€
   🎯 'Magic Discount' Discovery:
   • Tiny discounts (0-0.1%) dramatically increase conversion
   • Psychological threshold effect
   ACTION: Implement 'courtesy discount' of 0.1-0.5% for all quotes

3. SWITCHER CUSTOMERS (From earlier analysis):
   ⭐ HIGHEST VALUE SEGMENT
   • 50.8% conversion rate (vs 41.8% average)
   • 4,295€ average margin (vs 2,734€)
   • 61% choose heat pumps when guided
   ACTION: Prioritize customers considering both products

4. SALES CYCLE MANAGEMENT:
   ⏱️ Heat pumps take 12.9 days longer to close
   • But generate 216